In [67]:
import pandas as pd
import numpy as np
import sys
sys.path.append('../')
import pandas as pd
from src.gpt_prompt import get_sentiment_general, get_sentiment_general_parallel, get_risk_score
import datetime

In [68]:
df_news = pd.read_csv("../data/apple_2023/news_apple_2023.csv", index_col=0)
# df_news["date"] = pd.to_datetime(df_news['date'])

In [69]:
n = df_news.shape[0]
ind = 6
batch_size = 40

In [70]:
scores = []
batch = []
index = []
for i in range(n):
    if i % 10 == ind:
        index.append(i)
        batch.append(df_news["news"][i])
    # index.append(i)
    # batch.append(df_news["news"][i])
    if len(batch) >= batch_size:
        result1 = get_sentiment_general_parallel(batch,company="Apple",model="gpt-3.5-turbo-1106", output="polar", explanation=False, threads=20)
        print(f"finish {i}")
        scores += result1
        batch = []  
        # try:
        #     result1 = get_sentiment_general_parallel(batch,company="Apple",model="gpt-3.5-turbo-1106", output="polar", explanation=False, threads=20)
        #     print(f"finish {i}")
        #     scores += result1
        #     batch = []  
        # except:
        #     print(f"Fail {index[-1]}")
        #     index = index[:-batch_size]
        #     batch = [] 
            
            
if batch:
    result1 = get_sentiment_general_parallel(batch,company="Apple",model="gpt-3.5-turbo-1106", output="polar", explanation=False, threads=20)
    print(f"finish {i}")
    scores += result1
    batch = []  
    # try:
    #     result1 = get_sentiment_general_parallel(batch,company="Apple",model="gpt-3.5-turbo-1106", output="polar", explanation=False, threads=20)
    #     print(f"finish {i}")
    #     scores += result1
    #     batch = []  
    # except:
    #     print(f"Fail {index[-1]}")

        
    
    
    

 average running time: 0.18 second
finish 396
 average running time: 1.34 second
finish 796
 average running time: 1.93 second
finish 1196
 average running time: 0.16 second
finish 1596
 average running time: 0.14 second
finish 1996
 average running time: 0.15 second
finish 2396
 average running time: 0.17 second
finish 2796


ServiceUnavailableError: The server is overloaded or not ready yet.

In [78]:
def convert_response_to_score(response):
    dic = {"BULLISH": 1, "NEUTRAL": 0, "BEARISH":-1}
    scores = []
    for r in response:
        if r[0] not in dic:
            scores.append(5)
            continue
        scores.append(dic[r[0]])
        
    return scores

scores = convert_response_to_score(scores)
        

In [79]:
index = index[:len(scores)]
new_df = df_news.iloc[index].copy()
new_df["score"] = scores
new_df = new_df.reset_index(drop=True)
new_df.to_csv(f"../data/apple_2023/news_score_apple_2023_{ind+1}.csv")

In [80]:
new_df

,date,news,score
0,2023-01-01 05:07:26,Title: The Brits Are Coming: Five Rising Write...,0
1,2023-01-01 10:41:41,Title: OLED iPad Pro could get a whole new siz...,0
2,2023-01-02 02:05:30,Title: These are the electric cars we're most ...,1
3,2023-01-02 07:52:28,Title: Should Goldman Sachs ActiveBeta U.S. La...,0
4,2023-01-02 12:19:31,Title: The 10 Best Stocks to Buy in January 20...,1
...,...,...,...
275,2023-01-24 17:00:21,Title: Penbrook Management LLC's Top 5 Sells o...,0
276,2023-01-24 18:56:36,Title: Verizon Communications Inc. stock outpe...,1
277,2023-01-24 19:00:49,Title: Discover Financial Services stock outpe...,1
278,2023-01-24 19:03:10,Title: Nokia Corp. ADR underperforms Tuesday w...,-1


In [81]:
dictionary = new_df.set_index('date')['score'].to_dict()
dictionary

{'2023-01-01 05:07:26': 0,
 '2023-01-01 10:41:41': 0,
 '2023-01-02 02:05:30': 1,
 '2023-01-02 07:52:28': 0,
 '2023-01-02 12:19:31': 1,
 '2023-01-02 20:37:21': 1,
 '2023-01-03 04:04:28': 0,
 '2023-01-03 06:10:31': 1,
 '2023-01-03 07:38:22': -1,
 '2023-01-03 08:46:20': 0,
 '2023-01-03 09:26:14': 1,
 '2023-01-03 10:07:07': 1,
 '2023-01-03 11:06:34': -1,
 '2023-01-03 12:15:22': 1,
 '2023-01-03 13:00:15': 0,
 '2023-01-03 13:58:31': 1,
 '2023-01-03 15:01:47': 1,
 '2023-01-03 16:11:40': 1,
 '2023-01-03 17:26:22': 0,
 '2023-01-03 18:24:55': 0,
 '2023-01-03 20:13:24': -1,
 '2023-01-03 21:01:38': 1,
 '2023-01-03 21:06:15': 1,
 '2023-01-04 00:34:23': 1,
 '2023-01-04 03:11:31': -1,
 '2023-01-04 05:33:31': 1,
 '2023-01-04 06:19:28': -1,
 '2023-01-04 06:56:01': -1,
 '2023-01-04 07:39:17': 1,
 '2023-01-04 08:33:18': 1,
 '2023-01-04 09:04:54': 1,
 '2023-01-04 10:03:40': 1,
 '2023-01-04 10:33:22': 1,
 '2023-01-04 11:35:45': 1,
 '2023-01-04 12:32:16': 1,
 '2023-01-04 13:45:30': 1,
 '2023-01-04 14:59:38'

In [82]:
def find_whole_minute_time_after_5_minutes(time):
    new_time = given_time + timedelta(minutes=5)

    # Adjust to the nearest whole minute
    if new_time.second > 0:
        new_time += timedelta(minutes=1)
        new_time = new_time.replace(second=0, microsecond=0)

In [83]:
# news = df["news"][0]
# print(get_sentiment_general(news,company="Apple",model="gpt-3.5-turbo-16k", output="polar", explanation=False)[0])
# print(get_sentiment_general(news,company="Apple",model="gpt-3.5-turbo-1106", output="polar", explanation=False)[0])

In [84]:
# news = df["news"][::400]
# result1 = get_sentiment_general_parallel(news,company="Apple",model="gpt-3.5-turbo-16k", output="polar", explanation=False)
# print(result1)
# result2 = get_sentiment_general_parallel(news,company="Apple",model="gpt-3.5-turbo-1106", output="polar", explanation=False)
# print(result2)


In [85]:
# n = len(result1)
# c = 0
# for i in range(n):
#     if result1[i] == result2[i]:
#         c += 1
# print(c / n)
        